In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN1024"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 6    #number of convolution to have in the network
pooling_stride = [4, 2, 2, 2, 1, 1]
weights_shape = [[7, n_variables, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [3, 32, 32]]
bias_shape = [32, 32, 32, 32, 32, 32]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [7]:
print pos_weight

3.18601822124


In [9]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [10]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [11]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [12]:
model.build_conv_net(pos_weight = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [13]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [14]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.163541, fmeasure_max= 0.323686, precision= 0.196850, recall= 0.910064
Train Random guess:  , auc= 0.190903fmeasure_max= 0.309943, precision= 0.183465, recall= 0.997859
Validation Iter 0, auc= 0.233182, fmeasure_max= 0.371101, precision= 0.227823, recall= 1.000000
Validation Random guess:  auc= 0.245723, fmeasure_max= 0.371811, precision= 0.228359, recall= 1.000000
Minibatch Loss= 0.977947
Tarin Iter 100, auc= 0.733306, fmeasure_max= 0.695881, precision= 0.725777, recall= 0.668350
Train Random guess:  , auc= 0.227191fmeasure_max= 0.378622, precision= 0.234076, recall= 0.989899
Validation Iter 100, auc= 0.702446, fmeasure_max= 0.683333, precision= 0.742455, recall= 0.632933
Validation Random guess:  auc= 0.229552, fmeasure_max= 0.374499, precision= 0.232491, recall= 0.962264
Minibatch Loss= 0.887553
Tarin Iter 200, auc= 0.790345, fmeasure_max= 0.781173, precision= 0.815517, recall= 0.749604
Train Random guess:  , auc= 0.252950fmeasure_max= 0.396232, precision= 0.2470

In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [15]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.881329 F1_max = 0.799304 prec = 0.823656 rec = 0.776351
Test Random guess: auc= 0.234047 fmeasure_max= 0.375798 , precision= 0.231410 , recall= 0.999324


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})